# Polymarket Integration - WebSocket, Markets & Arbitrage Detection

**Prerequisites:** Run `02_complete_testing.ipynb` first!

**What we'll test:**
1. ✅ Generate Polymarket API credentials
2. ✅ Initialize Polymarket client
3. ✅ Fetch available markets
4. ✅ Get order book data
5. ✅ Calculate arbitrage opportunities
6. ✅ Test fee calculations

**Run each cell sequentially!**

---
## Section 1: Setup & Imports

In [1]:
print("🔍 Starting Polymarket Integration Testing...")
print("="*60)
print("")

print("📦 Importing libraries...")
import sys
import os
import asyncio
import time
from datetime import datetime
from pathlib import Path
from dotenv import load_dotenv

# Web3 & Blockchain
from web3 import Web3

# Polymarket SDK
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import ApiCreds
from py_clob_client.constants import POLYGON

# Speed optimizations
import orjson

# Data analysis
import pandas as pd
import numpy as np

print("✅ All imports successful!")
print("")
print("✅ Section 1 complete!")

🔍 Starting Polymarket Integration Testing...

📦 Importing libraries...
✅ All imports successful!

✅ Section 1 complete!


---
## Section 2: Load Configuration

In [2]:
print("🔍 Loading configuration...")
print("="*60)
print("")

# Load .env file
project_root = Path().absolute().parent
env_path = project_root / '.env'
load_dotenv(env_path)

# Get config
POLYGON_RPC_URL = os.getenv('POLYGON_RPC_URL')
WALLET_ADDRESS = os.getenv('WALLET_ADDRESS')
WALLET_PRIVATE_KEY = os.getenv('WALLET_PRIVATE_KEY')
DRY_RUN_MODE = os.getenv('DRY_RUN_MODE', 'true').lower() == 'true'
POLYMARKET_HOST = os.getenv('POLYMARKET_HOST', 'https://clob.polymarket.com')

print("📊 Configuration:")
print(f"   💰 Wallet: {WALLET_ADDRESS}")
print(f"   🧪 Dry Run: {DRY_RUN_MODE}")
print(f"   🌐 Polymarket Host: {POLYMARKET_HOST}")
print("")
print("✅ Section 2 complete!")

🔍 Loading configuration...

📊 Configuration:
   💰 Wallet: 0x03a5aC77e0497a90e7F155728d0Fb54F8A9A9F0B
   🧪 Dry Run: True
   🌐 Polymarket Host: https://clob.polymarket.com

✅ Section 2 complete!


---
## Section 3: Generate Polymarket API Credentials

**IMPORTANT:** This creates API keys for Polymarket CLOB.
- Keys are deterministically derived from your wallet's private key
- Same wallet = same API keys every time
- No need to store separately (can regenerate anytime)

In [3]:
print("🔍 Generating Polymarket API credentials...")
print("="*60)
print("")

# Check if we have private key
if not WALLET_PRIVATE_KEY or WALLET_PRIVATE_KEY == '0xYourPrivateKeyHere':
    print("❌ ERROR: WALLET_PRIVATE_KEY not set in .env file!")
    print("")
    print("⚠️ You need your private key to generate API credentials.")
    print("")
    print("📋 Options:")
    print("   1. Add your private key to .env file (WALLET_PRIVATE_KEY)")
    print("   2. Export from MetaMask: Account Details > Export Private Key")
    print("   3. NEVER share your private key with anyone!")
    print("")
    print("⚠️ Skipping credential generation for now...")
    api_creds_ready = False
else:
    try:
        print("⏱️ Validating private key format...")
        
        # Validate private key format
        if not WALLET_PRIVATE_KEY.startswith('0x'):
            print("❌ ERROR: Private key must start with '0x'")
            print("   Example: 0x1234567890abcdef...")
            api_creds_ready = False
        elif len(WALLET_PRIVATE_KEY) != 66:  # 0x + 64 hex chars
            print(f"⚠️ WARNING: Private key length is {len(WALLET_PRIVATE_KEY)}, expected 66")
            print("   Should be: 0x + 64 hexadecimal characters")
            api_creds_ready = False
        else:
            print("✅ Private key format valid!")
            print("")
            print("💡 Note: ClobClient will auto-generate API credentials")
            print("   from your private key in the next section")
            print("")
            print("🔑 Credentials are deterministically derived:")
            print("   - Same wallet = same API keys every time")
            print("   - No need to store separately")
            print("   - Handled automatically by py-clob-client")
            api_creds_ready = True
        
    except Exception as e:
        print(f"❌ ERROR: {str(e)}")
        import traceback
        print(traceback.format_exc())
        api_creds_ready = False

print("")
print("✅ Section 3 complete!")

🔍 Generating Polymarket API credentials...

⏱️ Validating private key format...
✅ Private key format valid!

💡 Note: ClobClient will auto-generate API credentials
   from your private key in the next section

🔑 Credentials are deterministically derived:
   - Same wallet = same API keys every time
   - No need to store separately
   - Handled automatically by py-clob-client

✅ Section 3 complete!


---
## Section 4: Initialize Polymarket Client

In [4]:
print("🔍 Initializing Polymarket CLOB client...")
print("="*60)
print("")

if not api_creds_ready:
    print("❌ Cannot initialize client - no valid private key")
    print("   Add your private key to .env and run Section 3 again")
    client = None
else:
    try:
        print("⏱️ Creating ClobClient with private key...")
        
        # Initialize client directly with private key
        # The library will auto-generate API credentials internally
        client = ClobClient(
            host=POLYMARKET_HOST,
            key=WALLET_PRIVATE_KEY,  # Private key, not API key
            chain_id=POLYGON
        )
        
        print("✅ Client initialized!")
        print("")
        print("📊 Client info:")
        print(f"   🌐 Host: {POLYMARKET_HOST}")
        print(f"   🔗 Chain ID: {POLYGON}")
        print(f"   💰 Address: {WALLET_ADDRESS}")
        print("")
        print("💡 API credentials auto-generated by library")
        
    except Exception as e:
        print(f"❌ ERROR: {str(e)}")
        print("")
        import traceback
        print(traceback.format_exc())
        client = None

print("")
print("✅ Section 4 complete!")

🔍 Initializing Polymarket CLOB client...

⏱️ Creating ClobClient with private key...
✅ Client initialized!

📊 Client info:
   🌐 Host: https://clob.polymarket.com
   🔗 Chain ID: 137
   💰 Address: 0x03a5aC77e0497a90e7F155728d0Fb54F8A9A9F0B

💡 API credentials auto-generated by library

✅ Section 4 complete!


---
## Section 5: Fetch Available Markets

Get a list of active markets on Polymarket

In [ ]:
# Cell 10: WebSocket Order Book Streaming (README-Aligned)
# Purpose: Stream real-time order book updates for high-volume markets
# Architecture: Matches README Section - "STREAMING LAYER"

import json
import time
import threading
from websocket import WebSocketApp
from datetime import datetime

print("\n" + "="*60)
print("📡 WEBSOCKET ORDER BOOK STREAMING - README ALIGNED")
print("="*60)

# CONFIGURATION
WS_URL = "wss://ws-subscriptions-clob.polymarket.com/ws/market"
MIN_EDGE = 0.006  # 0.6% - profitable after fees
PROFIT_THRESHOLD = 0.01  # 1% minimum profit
CHECK_INTERVAL_SEC = 1.0  # Check for arbs every second

print(f"\n⚙️ Configuration:")
print(f"   WebSocket: {WS_URL}")
print(f"   Min edge (profitable): {MIN_EDGE*100:.2f}%")
print(f"   Profit threshold: {PROFIT_THRESHOLD*100:.2f}%")
print(f"\n💡 Strategy: Monitor high-volume markets for arbitrage opportunities")
print(f"   Per README: STREAMING LAYER → STRATEGY LAYER → (future: EXECUTION)\n")

# ============================================================================
# STEP 1: Select Markets to Monitor (from Cell 14's data)
# ============================================================================

print("="*60)
print("🎯 STEP 1: Select Markets to Monitor")
print("="*60)

# Check if we have markets from Cell 14
if 'markets' not in globals() or not markets or len(markets) == 0:
    print("\n❌ ERROR: No markets available!")
    print("   Please run Cell 14 first to fetch market data.\n")
    raise RuntimeError("Markets not loaded. Run Cell 14 first.")

# Select top 3 high-volume BINARY markets
print(f"\nScanning {len(markets)} markets for high-volume binary markets...\n")

monitored_markets = []

for market in markets[:50]:  # Check first 50
    try:
        # Parse token data
        clob_tokens_str = market.get('clobTokenIds', '[]')
        outcomes_str = market.get('outcomes', '[]')

        clob_tokens = json.loads(clob_tokens_str)
        outcomes = json.loads(outcomes_str)

        # Only binary markets
        if len(clob_tokens) != 2:
            continue

        volume = float(market.get('volume24hr', 0) or 0)
        question = market.get('question', 'Unknown')

        monitored_markets.append({
            'question': question,
            'volume': volume,
            'tokens': clob_tokens,
            'outcomes': outcomes,
            'condition_id': market.get('conditionId'),
        })

        print(f"✅ Selected: {question[:60]}...")
        print(f"   Volume: ${volume:,.0f}")
        print(f"   Tokens: {clob_tokens[0][:20]}... / {clob_tokens[1][:20]}...")
        print()

        if len(monitored_markets) >= 3:  # Monitor top 3
            break

    except Exception as e:
        continue

if len(monitored_markets) == 0:
    print("\n❌ No binary markets found!")
    raise RuntimeError("No suitable markets to monitor")

print(f"{'='*60}")
print(f"✅ Monitoring {len(monitored_markets)} markets")
print(f"{'='*60}\n")

# ============================================================================
# STEP 2: WebSocket Client with Order Book Management
# ============================================================================

print("="*60)
print("📡 STEP 2: Initialize WebSocket Client")
print("="*60)

class OrderBookStreamer:
    def __init__(self, markets):
        self.markets = markets
        self.ws = None
        self.connected = False
        self.last_activity = time.time()
        self.message_count = 0

        # Order book state (key = token_id)
        self.orderbooks = {}

        # Initialize order book structure for each token
        for market in self.markets:
            for token_id in market['tokens']:
                self.orderbooks[token_id] = {
                    'bids': [],
                    'asks': [],
                    'last_update': None
                }

        # Arbitrage tracking
        self.opportunities_found = 0
        self.last_check_time = time.time()

        print(f"\n✅ Initialized order book state for {len(self.orderbooks)} tokens")

    def on_open(self, ws):
        print(f"\n{'='*60}")
        print("✅ WebSocket Connected!")
        print(f"{'='*60}\n")

        self.connected = True

        # Subscribe to all token IDs
        all_token_ids = []
        for market in self.markets:
            all_token_ids.extend(market['tokens'])

        subscription = {
            "assets_ids": all_token_ids,
            "type": "market"
        }

        print(f"📡 Subscribing to {len(all_token_ids)} token order books...")
        ws.send(json.dumps(subscription))
        print(f"✅ Subscription sent!\n")

        # Start background tasks
        self.start_ping_thread()
        self.start_arbitrage_checker()

    def on_message(self, ws, message):
        self.last_activity = time.time()
        self.message_count += 1

        try:
            data = json.loads(message)
            event_type = data.get("event_type")

            # Progress indicator
            if self.message_count % 20 == 0:
                print(f"📨 {self.message_count} messages received...")

            # Handle order book updates
            if event_type == "book":
                asset_id = data.get("asset_id")

                if asset_id in self.orderbooks:
                    # Update order book state
                    self.orderbooks[asset_id]['bids'] = data.get('bids', [])
                    self.orderbooks[asset_id]['asks'] = data.get('asks', [])
                    self.orderbooks[asset_id]['last_update'] = time.time()

            elif event_type == "price_change":
                # Handle price change events
                for change in data.get("price_changes", []):
                    asset_id = change.get("asset_id")
                    if asset_id in self.orderbooks:
                        # Update with new prices
                        best_bid = change.get("best_bid")
                        best_ask = change.get("best_ask")

                        if best_bid:
                            self.orderbooks[asset_id]['bids'] = [[best_bid, 0]]
                        if best_ask:
                            self.orderbooks[asset_id]['asks'] = [[best_ask, 0]]

                        self.orderbooks[asset_id]['last_update'] = time.time()

        except Exception as e:
            if self.message_count <= 5:
                print(f"⚠️  Error parsing message: {str(e)[:50]}")

    def check_arbitrage_opportunities(self):
        """
        Check all markets for arbitrage opportunities
        This is the STRATEGY LAYER from README
        """
        for market in self.markets:
            try:
                token1, token2 = market['tokens']

                # Get order books
                ob1 = self.orderbooks.get(token1)
                ob2 = self.orderbooks.get(token2)

                if not ob1 or not ob2:
                    continue

                # Check if we have fresh data
                if not ob1['asks'] or not ob2['asks']:
                    continue

                # Get best ask prices (price to BUY)
                ask1 = float(ob1['asks'][0][0])  # [price, size]
                ask2 = float(ob2['asks'][0][0])

                # Calculate arbitrage
                total_cost = ask1 + ask2
                gross_edge = 1.0 - total_cost

                # Estimate costs
                trade_size = 100.0
                fees = trade_size * 0.002 * 2  # 0.2% taker on both sides
                gas = 0.05
                slippage = trade_size * 0.001
                total_costs = fees + gas + slippage
                cost_pct = total_costs / trade_size

                net_edge = gross_edge - cost_pct

                # Check if profitable
                if net_edge > PROFIT_THRESHOLD:
                    self.opportunities_found += 1

                    print(f"\n{'='*60}")
                    print(f"🎉 ARBITRAGE OPPORTUNITY #{self.opportunities_found}")
                    print(f"{'='*60}")
                    print(f"\n📊 Market: {market['question'][:60]}...")
                    print(f"\n💰 Prices:")
                    print(f"   {market['outcomes'][0]}: ${ask1:.4f}")
                    print(f"   {market['outcomes'][1]}: ${ask2:.4f}")
                    print(f"   Total cost: ${total_cost:.4f}")
                    print(f"\n📈 Edge Analysis:")
                    print(f"   Gross edge: {gross_edge*100:.2f}%")
                    print(f"   Costs: {cost_pct*100:.2f}%")
                    print(f"   Net edge: {net_edge*100:.2f}%")
                    print(f"   Profit per $100: ${net_edge*100:.2f}")
                    print(f"\n⏰ Time: {datetime.now().strftime('%H:%M:%S')}")
                    print(f"{'='*60}\n")

            except Exception as e:
                continue

    def start_arbitrage_checker(self):
        """Background thread to check for arbitrage periodically"""
        def checker_loop():
            while self.connected:
                try:
                    current_time = time.time()
                    if current_time - self.last_check_time >= CHECK_INTERVAL_SEC:
                        self.check_arbitrage_opportunities()
                        self.last_check_time = current_time

                    time.sleep(0.1)  # Small sleep to avoid spinning
                except Exception as e:
                    print(f"⚠️  Arbitrage checker error: {str(e)[:50]}")
                    time.sleep(1)

        thread = threading.Thread(target=checker_loop, daemon=True)
        thread.start()
        print("✅ Arbitrage checker started (runs every 1 second)\n")

    def start_ping_thread(self):
        """Keep WebSocket connection alive"""
        def ping_loop():
            while self.connected:
                try:
                    # Send ping if no activity for 30 seconds
                    if time.time() - self.last_activity > 30:
                        self.ws.send(json.dumps({"type": "ping"}))
                    time.sleep(30)
                except Exception as e:
                    break

        thread = threading.Thread(target=ping_loop, daemon=True)
        thread.start()
        print("✅ Ping thread started (keeps connection alive)\n")

    def on_error(self, ws, error):
        print(f"\n❌ WebSocket Error: {error}\n")

    def on_close(self, ws, close_status_code, close_msg):
        print(f"\n🔌 WebSocket Closed (code: {close_status_code})")
        print(f"   Message: {close_msg}\n")
        self.connected = False

    def start(self):
        """Start the WebSocket connection"""
        self.ws = WebSocketApp(
            WS_URL,
            on_open=self.on_open,
            on_message=self.on_message,
            on_error=self.on_error,
            on_close=self.on_close
        )

        # Run in background thread
        ws_thread = threading.Thread(target=self.ws.run_forever, daemon=True)
        ws_thread.start()

        return ws_thread

# ============================================================================
# STEP 3: Start Streaming
# ============================================================================

print("\n{'='*60}")
print("🚀 STEP 3: Start WebSocket Streaming")
print("="*60)

streamer = OrderBookStreamer(monitored_markets)
ws_thread = streamer.start()

print("\n⏳ Waiting for connection...")
time.sleep(3)

if streamer.connected:
    print("\n{'='*60}")
    print("✅ STREAMING ACTIVE!")
    print("="*60)
    print(f"\n📡 Monitoring {len(monitored_markets)} markets in real-time")
    print(f"💰 Looking for arbitrage opportunities > {PROFIT_THRESHOLD*100:.1f}%")
    print(f"\n⚠️  Press 'Stop' button or Ctrl+C to stop streaming\n")
    print(f"{'='*60}\n")

    # Run for a while (or until user stops)
    try:
        while True:
            time.sleep(5)

            # Status update every 30 seconds
            if streamer.message_count > 0 and streamer.message_count % 150 == 0:
                print(f"\n📊 STATUS UPDATE:")
                print(f"   Messages: {streamer.message_count}")
                print(f"   Opportunities found: {streamer.opportunities_found}")
                print(f"   Connected: {'🟢' if streamer.connected else '🔴'}")
                print(f"   Time: {datetime.now().strftime('%H:%M:%S')}\n")

    except KeyboardInterrupt:
        print(f"\n\n⚠️  Streaming stopped by user")
        print(f"\n📊 Final Statistics:")
        print(f"   Total messages: {streamer.message_count}")
        print(f"   Opportunities found: {streamer.opportunities_found}")
        print(f"   Runtime: {(time.time() - streamer.last_activity)/60:.1f} minutes\n")
else:
    print("\n❌ Failed to connect to WebSocket")
    print("   Check network connection and try again\n")

print("✅ Section 5 complete!")

In [ ]:
# Section 5+6: INFINITE SEARCH MODE - Finds 1 opportunity then stops

import requests
import json
import time

print("\n" + "="*60)
print("🔍 INFINITE ARBITRAGE SEARCH")
print("="*60)

GAMMA_API = "https://gamma-api.polymarket.com"
MIN_EDGE = 0.002       # 0.2% - very low threshold
PAGE_SIZE = 100
DELAY = 0.3            # Seconds between pages (be nice to API)

print(f"\n⚙️ Search Parameters:")
print(f"   Target: Find 1 arbitrage opportunity")
print(f"   Min edge: {MIN_EDGE*100:.2f}%")
print(f"   Max pages: UNLIMITED (will search forever)")
print(f"   Rate limit delay: {DELAY}s between pages")
print(f"\n⚠️  Press Ctrl+C or stop cell to abort\n")

checked_count = 0
page = 0
start_time = time.time()

print("🔎 Starting infinite search...\n")

while True:  # ← INFINITE LOOP
    try:
        offset = page * PAGE_SIZE
        elapsed = time.time() - start_time
        
        # Progress indicator every 5 pages
        if page % 5 == 0:
            print(f"{'='*60}")
            print(f"📊 PROGRESS UPDATE")
            print(f"{'='*60}")
            print(f"   Pages scanned: {page + 1}")
            print(f"   Markets checked: {checked_count}")
            print(f"   Time elapsed: {elapsed/60:.1f} minutes")
            print(f"   Rate: {checked_count/(elapsed/60):.0f} markets/min")
            print()
        
        print(f"📄 Page {page + 1}: Fetching markets {offset+1}-{offset+PAGE_SIZE}...")
        
        response = requests.get(
            f"{GAMMA_API}/markets",
            params={
                'closed': 'false',
                'active': 'true',
                'limit': PAGE_SIZE,
                'offset': offset,
                'order': 'volume24hr',
                'ascending': 'false'
            },
            timeout=10
        )
        
        # Check for rate limiting
        if response.status_code == 429:
            print(f"   ⚠️  Rate limited! Waiting 60 seconds...")
            time.sleep(60)
            continue
        
        response.raise_for_status()
        markets_page = response.json()
        
        if not markets_page or len(markets_page) == 0:
            print(f"   ℹ️  Reached end of markets. Restarting from beginning...")
            page = 0
            time.sleep(5)
            continue
        
        print(f"   ✓ Retrieved {len(markets_page)} markets")
        
        # Check each market
        found = False
        for market in markets_page:
            checked_count += 1
            
            try:
                question = market.get('question', 'Unknown')[:55]
                
                # Parse data
                clob_token_ids_str = market.get('clobTokenIds', '[]')
                outcomes_str = market.get('outcomes', '[]')
                prices_str = market.get('outcomePrices', '[]')
                
                market_tokens = json.loads(clob_token_ids_str)
                market_outcomes = json.loads(outcomes_str)
                market_prices = json.loads(prices_str)
                
                # Binary only
                if len(market_tokens) != 2 or len(market_prices) != 2:
                    continue
                
                # Calculate edge
                try:
                    price1 = float(market_prices[0])
                    price2 = float(market_prices[1])
                except (ValueError, TypeError):
                    continue
                
                total_cost = price1 + price2
                gross_edge = 1.0 - total_cost
                
                # Check if viable
                if gross_edge > MIN_EDGE:
                    # FOUND ONE!
                    print(f"\n{'='*60}")
                    print(f"🎉 ARBITRAGE FOUND!")
                    print(f"{'='*60}\n")
                    
                    selected_market = market
                    token_ids = market_tokens
                    outcomes = market_outcomes
                    prices = market_prices
                    
                    print(f"📊 Market:")
                    print(f"   {question}")
                    print(f"   Volume: ${float(market.get('volume24hr', 0)):,.0f}")
                    print(f"   End: {market.get('endDate', 'N/A')}")
                    print(f"   Slug: {market.get('slug', 'N/A')}")
                    
                    print(f"\n🎯 Prices:")
                    print(f"   {market_outcomes[0]}: ${price1:.4f} ({price1*100:.1f}%)")
                    print(f"   {market_outcomes[1]}: ${price2:.4f} ({price2*100:.1f}%)")
                    print(f"   Total: ${total_cost:.4f}")
                    
                    print(f"\n💰 Edge Analysis:")
                    print(f"   Gross edge: ${gross_edge:.4f} ({gross_edge*100:.2f}%)")
                    
                    # Cost calc
                    trade_size = 100
                    fees = trade_size * 0.004
                    gas = 0.05
                    total_costs = fees + gas
                    cost_pct = total_costs / trade_size
                    net_edge = gross_edge - cost_pct
                    
                    print(f"   Est. costs: ${total_costs:.2f} ({cost_pct*100:.2f}%)")
                    print(f"   Net edge: ${gross_edge - cost_pct:.4f} ({net_edge*100:.2f}%)")
                    
                    if net_edge > 0:
                        print(f"\n   ✅ PROFITABLE after fees!")
                        print(f"   💵 ${net_edge * trade_size:.2f} profit per $100 trade")
                    else:
                        print(f"\n   ⚠️  Not profitable after fees")
                    
                    print(f"\n📈 Search Statistics:")
                    print(f"   Total markets checked: {checked_count}")
                    print(f"   Pages scanned: {page + 1}")
                    print(f"   Time taken: {(time.time() - start_time)/60:.1f} minutes")
                    
                    found = True
                    break
                
            except Exception:
                continue
        
        if found:
            print(f"\n✅ Search complete! Found 1 opportunity.")
            break
        
        # Brief summary
        print(f"   Checked {len([m for m in markets_page if len(json.loads(m.get('outcomes','[]'))) == 2])} binary markets")
        print()
        
        page += 1
        
        # Rate limiting delay
        time.sleep(DELAY)
        
    except KeyboardInterrupt:
        print(f"\n\n⚠️  Search interrupted by user")
        print(f"   Checked {checked_count} markets across {page + 1} pages")
        print(f"   Time: {(time.time() - start_time)/60:.1f} minutes")
        print(f"\n   No arbitrage found at {MIN_EDGE*100:.2f}% threshold")
        
        token_ids = None
        selected_market = None
        outcomes = None
        prices = None
        break
        
    except requests.exceptions.RequestException as e:
        print(f"\n❌ API Error: {e}")
        print(f"   Waiting 10 seconds before retry...")
        time.sleep(10)
        continue
        
    except Exception as e:
        print(f"\n❌ Unexpected error: {e}")
        import traceback
        print(traceback.format_exc())
        
        # Try to continue
        print(f"   Attempting to continue...")
        time.sleep(2)
        page += 1
        continue

print("\n✅ Section 5+6 complete!")

In [ ]:
# Diagnostic: Why are markets being rejected?

import requests
import json

print("\n" + "="*60)
print("🔍 DIAGNOSTIC: Market Rejection Analysis")
print("="*60)

GAMMA_API = "https://gamma-api.polymarket.com"

# Fetch first page
response = requests.get(
    f"{GAMMA_API}/markets",
    params={
        'closed': 'false',
        'active': 'true',
        'limit': 20,
        'offset': 0,
        'order': 'volume24hr',
        'ascending': 'false'
    },
    timeout=10
)
markets = response.json()

print(f"\nAnalyzing first 20 markets...\n")

stats = {
    'total': 0,
    'not_binary': 0,
    'no_tokens': 0,
    'parse_error': 0,
    'no_orderbook': 0,
    'no_liquidity': 0,
    'checked_edge': 0,
    'edge_too_small': 0
}

for idx, market in enumerate(markets[:20], 1):
    stats['total'] += 1
    
    question = market.get('question', 'Unknown')[:50]
    print(f"{idx}. {question}...")
    
    # Check 1: Parse tokens
    try:
        clob_token_ids_str = market.get('clobTokenIds', '[]')
        outcomes_str = market.get('outcomes', '[]')
        
        market_tokens = json.loads(clob_token_ids_str)
        market_outcomes = json.loads(outcomes_str)
        
        if not market_tokens or len(market_tokens) == 0:
            print(f"   ✗ No tokens found")
            stats['no_tokens'] += 1
            continue
            
    except Exception as e:
        print(f"   ✗ Parse error: {str(e)[:30]}")
        stats['parse_error'] += 1
        continue
    
    # Check 2: Binary only
    if len(market_tokens) != 2:
        print(f"   ✗ Not binary ({len(market_tokens)} outcomes)")
        stats['not_binary'] += 1
        continue
    
    print(f"   ✓ Binary market: {market_outcomes}")
    
    # Check 3: Fetch orderbooks
    try:
        ob1 = client.get_order_book(market_tokens[0])
        ob2 = client.get_order_book(market_tokens[1])
        print(f"   ✓ Orderbooks fetched")
    except Exception as e:
        print(f"   ✗ Orderbook error: {str(e)[:40]}")
        stats['no_orderbook'] += 1
        continue
    
    # Check 4: Liquidity
    if len(ob1.asks) == 0 or len(ob2.asks) == 0:
        print(f"   ✗ No liquidity (asks: {len(ob1.asks)}, {len(ob2.asks)})")
        stats['no_liquidity'] += 1
        continue
    
    print(f"   ✓ Has liquidity")
    
    # Check 5: Calculate edge
    try:
        ask1 = float(ob1.asks[0].price)
        ask2 = float(ob2.asks[0].price)
        total_cost = ask1 + ask2
        gross_edge = 1.0 - total_cost
        
        stats['checked_edge'] += 1
        
        print(f"   📊 Prices:")
        print(f"      {market_outcomes[0]}: ${ask1:.4f}")
        print(f"      {market_outcomes[1]}: ${ask2:.4f}")
        print(f"      Total: ${total_cost:.4f}")
        print(f"      Edge: {gross_edge*100:.2f}%")
        
        if gross_edge > 0.006:
            print(f"   ✅ VIABLE! (edge > 0.6%)")
        else:
            print(f"   ✗ Edge too small (need >0.6%)")
            stats['edge_too_small'] += 1
    
    except Exception as e:
        print(f"   ✗ Edge calc error: {str(e)[:40]}")
        continue
    
    print()

print("="*60)
print("📊 SUMMARY")
print("="*60)
print(f"\nTotal markets analyzed: {stats['total']}")
print(f"\nRejection reasons:")
print(f"   Not binary: {stats['not_binary']}")
print(f"   No tokens: {stats['no_tokens']}")
print(f"   Parse errors: {stats['parse_error']}")
print(f"   No orderbook: {stats['no_orderbook']}")
print(f"   No liquidity: {stats['no_liquidity']}")
print(f"   Edge calculated: {stats['checked_edge']}")
print(f"   Edge too small (<0.6%): {stats['edge_too_small']}")

print(f"\n💡 Insights:")
if stats['not_binary'] > stats['total'] * 0.5:
    print(f"   - Most markets are multi-outcome (not binary)")
    print(f"   - Consider implementing multi-outcome arbitrage")
if stats['edge_too_small'] > 0:
    print(f"   - Found {stats['edge_too_small']} markets with edges, but too small")
    print(f"   - Try lowering MIN_EDGE threshold")
if stats['no_liquidity'] > 0:
    print(f"   - {stats['no_liquidity']} markets have no liquidity")
if stats['checked_edge'] == 0:
    print(f"   - No markets reached edge calculation stage")
    print(f"   - Problem is earlier in the pipeline")

In [ ]:
# Find Khamenei market in existing market data

import json

print("\n" + "="*60)
print("🔍 FINDING KHAMENEI IN EXISTING MARKETS")
print("="*60)

# We already have markets from Gamma API in the 'markets' variable
# Let's search for Khamenei

print(f"\nSearching through {len(markets)} markets for 'Khamenei'...\n")

khamenei_markets = []

for i, market in enumerate(markets):
    question = market.get('question', '')
    if 'khamenei' in question.lower():
        khamenei_markets.append(market)
        
        print(f"Found #{len(khamenei_markets)}: {question}")
        print(f"   Slug: {market.get('slug', 'N/A')}")
        print(f"   Condition ID: {market.get('conditionId', 'N/A')}")
        
        # Parse token data
        try:
            clob_tokens = json.loads(market.get('clobTokenIds', '[]'))
            outcomes = json.loads(market.get('outcomes', '[]'))
            outcome_prices = json.loads(market.get('outcomePrices', '[]'))
            
            print(f"   Outcomes: {outcomes}")
            print(f"   Prices from Gamma: {outcome_prices}")
            
            # Fetch orderbooks
            if len(clob_tokens) >= 2:
                print(f"\n   Fetching orderbooks...")
                
                ob_yes = client.get_order_book(clob_tokens[0])
                ob_no = client.get_order_book(clob_tokens[1])
                
                yes_ask = float(ob_yes.asks[0].price) if len(ob_yes.asks) > 0 else None
                yes_bid = float(ob_yes.bids[0].price) if len(ob_yes.bids) > 0 else None
                no_ask = float(ob_no.asks[0].price) if len(ob_no.asks) > 0 else None
                no_bid = float(ob_no.bids[0].price) if len(ob_no.bids) > 0 else None
                
                print(f"\n   📊 CLOB Orderbook:")
                print(f"      YES: ask=${yes_ask:.4f}, bid=${yes_bid:.4f}")
                print(f"      NO:  ask=${no_ask:.4f}, bid=${no_bid:.4f}")
                
                print(f"\n   💡 Price Interpretations:")
                
                # Direct
                if yes_ask and no_ask:
                    total = yes_ask + no_ask
                    edge = 1 - total
                    print(f"      Direct asks: ${total:.4f} (edge: {edge*100:.2f}%)")
                
                # Midpoint (likely what website shows)
                if yes_ask and yes_bid and no_ask and no_bid:
                    yes_mid = (yes_ask + yes_bid) / 2
                    no_mid = (no_ask + no_bid) / 2
                    print(f"      Midpoint: YES ${yes_mid:.4f}, NO ${no_mid:.4f}")
                    print(f"      Midpoint total: ${yes_mid + no_mid:.4f}")
                
                # From Gamma prices
                if outcome_prices and len(outcome_prices) >= 2:
                    gamma_yes = float(outcome_prices[0])
                    gamma_no = float(outcome_prices[1])
                    print(f"      Gamma prices: YES ${gamma_yes:.4f}, NO ${gamma_no:.4f}")
                    print(f"      Gamma total: ${gamma_yes + gamma_no:.4f}")
                
        except Exception as e:
            print(f"   ❌ Error: {str(e)[:50]}")
        
        print()

if not khamenei_markets:
    print("❌ No Khamenei markets found in the current batch")
    print("   The market might be on a later page")
else:
    print(f"\n✅ Found {len(khamenei_markets)} Khamenei-related markets")

print("\n" + "="*60)

In [ ]:
# Section 5: Fetch Active Markets (FIXED - Uses Gamma API)

import requests
import pandas as pd
from datetime import datetime

print("\n" + "="*60)
print("🔍 Fetching ACTIVE markets from Polymarket...")
print("="*60)

# Use Gamma API for active markets (NOT CLOB)
GAMMA_API = "https://gamma-api.polymarket.com"

print("\n⏱️ Fetching from Gamma API...")

try:
    # Request parameters for active markets only
    params = {
        'closed': 'false',      # Only open markets
        'active': 'true',       # Only active markets  
        'limit': 100,           # 100 markets per page
        'offset': 0,            # Start at beginning
        'order': 'volume24hr',  # Sort by volume
        'ascending': 'false'    # Highest first
    }
    
    response = requests.get(
        f"{GAMMA_API}/markets",
        params=params,
        timeout=10
    )
    response.raise_for_status()
    markets_data = response.json()
    
    print(f"✅ Received {len(markets_data)} markets from API")
    
    if len(markets_data) == 0:
        print("\n⚠️ No active markets found")
        markets = []
        df = pd.DataFrame()
    else:
        df = pd.DataFrame(markets_data)
        
        # Filter for CLOB-enabled markets if column exists
        if 'enableOrderBook' in df.columns:
            df_tradable = df[df['enableOrderBook'] == True].copy()
            print(f"✅ Markets with order books: {len(df_tradable)}")
        else:
            df_tradable = df.copy()
        
        # Show top markets
        print(f"\n🎯 Top 10 markets by volume:")
        for i, row in df_tradable.head(10).iterrows():
            question = row.get('question', 'Unknown')[:60]
            volume = row.get('volume24hr', row.get('volume', 0))
            print(f"   {i+1}. {question}... (${float(volume):,.0f})" if volume else f"   {i+1}. {question}...")
        
        # Final dataset
        markets = df_tradable.to_dict('records')
        df = pd.DataFrame(markets)
        print(f"\n✅ Ready: {len(df)} tradable markets")

except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    print(traceback.format_exc())
    markets = []
    df = pd.DataFrame()

print("\n✅ Section 5 complete!")

---
## Section 6: Select a Market to Monitor

Choose a high-volume market for testing

In [ ]:
# Section 6: Select Market and Extract Token IDs (FIXED - handles string arrays)

print("\n" + "="*60)
print("🔍 Selecting a market to monitor...")
print("="*60)

# Check if we have markets from Section 5
if not markets or len(markets) == 0:
    print("\n❌ No markets available!")
    print("   Please run Section 5 first to fetch markets.")
    token_ids = None
else:
    # Select the first market (highest volume from Section 5)
    selected_market = markets[0]
    
    print("\n✅ Selected market:")
    print(f"\n📊 Market Details:")
    print(f"   ❓ Question: {selected_market.get('question', 'N/A')}")
    print(f"   🆔 Condition ID: {selected_market.get('conditionId', 'N/A')}")
    print(f"   📅 End Date: {selected_market.get('endDate', selected_market.get('end_date_iso', 'N/A'))}")
    print(f"   ✅ Active: {selected_market.get('active', 'N/A')}")
    print(f"   🔒 Closed: {selected_market.get('closed', 'N/A')}")
    
    # Check for different possible token field names in Gamma API
    tokens_data = None
    token_ids = []
    
    # Try different field names
    if 'tokens' in selected_market and selected_market['tokens']:
        tokens_data = selected_market['tokens']
    elif 'outcomes' in selected_market and selected_market['outcomes']:
        tokens_data = selected_market['outcomes']
    elif 'clobTokenIds' in selected_market and selected_market['clobTokenIds']:
        tokens_data = selected_market['clobTokenIds']
    
    if tokens_data:
        print(f"\n✅ Found {len(tokens_data)} token(s)")
        
        # Check if tokens_data is a list of strings or list of objects
        if isinstance(tokens_data[0], str):
            # Simple array of token ID strings
            print("   Format: Direct token ID array")
            token_ids = tokens_data
            
            for i, token_id in enumerate(token_ids):
                print(f"   {i+1}. Token: {token_id}")
        
        elif isinstance(tokens_data[0], dict):
            # Array of token objects
            print("   Format: Token objects")
            
            for i, token in enumerate(tokens_data):
                # Try different field names for token ID
                token_id = (
                    token.get('token_id') or 
                    token.get('tokenId') or 
                    token.get('id') or
                    None
                )
                
                if token_id:
                    outcome = token.get('outcome') or token.get('name') or f"Outcome {i+1}"
                    price = token.get('price', 'N/A')
                    
                    token_ids.append(token_id)
                    print(f"   {i+1}. {outcome}: {token_id[:20]}...")
                    if price != 'N/A':
                        print(f"      Price: ${float(price):.4f}")
        else:
            print(f"   ⚠️ Unexpected token data type: {type(tokens_data[0])}")
        
        if not token_ids:
            print("\n⚠️ Could not extract any token IDs")
            token_ids = None
        elif len(token_ids) < 2:
            print(f"\n⚠️ Warning: Only found {len(token_ids)} token(s)")
            print("   Need at least 2 tokens (YES/NO) for arbitrage")
        else:
            print(f"\n✅ Successfully extracted {len(token_ids)} token IDs")
            
            # For binary markets (2 tokens), label them
            if len(token_ids) == 2:
                print("\n🎯 Token mapping:")
                print(f"   YES (buy to bet event happens): {token_ids[0][:20]}...")
                print(f"   NO (buy to bet event doesn't happen): {token_ids[1][:20]}...")
    else:
        print("\n⚠️ No token data found in market")
        print(f"   Available fields: {list(selected_market.keys())}")
        token_ids = None

print("\n✅ Section 6 complete!")

In [ ]:
# Section 6 - FINAL: Find markets with actual arbitrage opportunities

import json

print("\n" + "="*60)
print("🔍 Searching for arbitrage-viable markets...")
print("="*60)

if not markets or len(markets) == 0:
    print("\n❌ No markets available!")
    token_ids = None
    selected_market = None
else:
    print("\n⏱️ Scanning markets for arbitrage opportunities...")
    print("   (Checking orderbook spreads...)\n")
    
    selected_market = None
    token_ids = None
    outcomes = None
    prices = None
    
    # Check multiple markets to find one with arbitrage
    for idx, market in enumerate(markets[:20], 1):  # Check first 20 markets
        try:
            question = market.get('question', 'Unknown')[:60]
            
            # Parse token data
            clob_token_ids_str = market.get('clobTokenIds', '[]')
            outcomes_str = market.get('outcomes', '[]')
            prices_str = market.get('outcomePrices', '[]')
            
            market_tokens = json.loads(clob_token_ids_str)
            market_outcomes = json.loads(outcomes_str)
            market_prices = json.loads(prices_str)
            
            # Only check binary markets for now (simpler)
            if len(market_tokens) != 2:
                continue
            
            print(f"   {idx}. {question}...")
            
            # Fetch orderbooks
            ob1 = client.get_order_book(market_tokens[0])
            ob2 = client.get_order_book(market_tokens[1])
            
            # Check for liquidity
            if len(ob1.asks) == 0 or len(ob2.asks) == 0:
                print(f"      ✗ No liquidity")
                continue
            
            # Get best ask prices (price to BUY)
            ask1 = float(ob1.asks[0].price)
            ask2 = float(ob2.asks[0].price)
            
            # Calculate total cost to buy both
            total_cost = ask1 + ask2
            gross_edge = 1.0 - total_cost
            gross_edge_pct = gross_edge * 100
            
            # Check if there's an arbitrage opportunity
            # Need at least 0.6% edge to cover fees (~0.4%) + buffer
            min_edge = 0.006  # 0.6%
            
            if gross_edge > min_edge:
                # Found a viable opportunity!
                selected_market = market
                token_ids = market_tokens
                outcomes = market_outcomes
                prices = market_prices
                
                print(f"      ✅ ARBITRAGE FOUND!")
                print(f"         {market_outcomes[0]} ask: ${ask1:.4f}")
                print(f"         {market_outcomes[1]} ask: ${ask2:.4f}")
                print(f"         Total: ${total_cost:.4f}")
                print(f"         Edge: ${gross_edge:.4f} ({gross_edge_pct:.2f}%)")
                break
            else:
                print(f"      ✗ Edge too small: {gross_edge_pct:.2f}% (need >{min_edge*100:.1f}%)")
                
        except Exception as e:
            print(f"      ✗ Error: {str(e)[:40]}...")
            continue
    
    if selected_market and token_ids:
        print(f"\n{'='*60}")
        print("✅ SELECTED ARBITRAGE OPPORTUNITY:")
        print(f"{'='*60}")
        
        print(f"\n📊 Market Details:")
        print(f"   ❓ Question: {selected_market.get('question', 'N/A')}")
        print(f"   🆔 Condition ID: {selected_market.get('conditionId', 'N/A')}")
        print(f"   📅 End Date: {selected_market.get('endDate', 'N/A')}")
        print(f"   💰 24h Volume: ${float(selected_market.get('volume24hr', 0)):,.0f}")
        
        print(f"\n🎯 Outcomes and Token IDs:")
        for i, (outcome, token_id) in enumerate(zip(outcomes, token_ids)):
            print(f"   {i+1}. {outcome}: {token_id}")
        
        # Fetch fresh orderbooks for display
        ob1 = client.get_order_book(token_ids[0])
        ob2 = client.get_order_book(token_ids[1])
        
        ask1 = float(ob1.asks[0].price)
        ask2 = float(ob2.asks[0].price)
        bid1 = float(ob1.bids[0].price) if len(ob1.bids) > 0 else 0
        bid2 = float(ob2.bids[0].price) if len(ob2.bids) > 0 else 0
        
        print(f"\n📈 Current Orderbook Prices:")
        print(f"   {outcomes[0]}: Ask ${ask1:.4f}, Bid ${bid1:.4f}")
        print(f"   {outcomes[1]}: Ask ${ask2:.4f}, Bid ${bid2:.4f}")
        
        total_cost = ask1 + ask2
        gross_edge = 1.0 - total_cost
        
        print(f"\n💰 Arbitrage Analysis:")
        print(f"   Total cost (buy both): ${total_cost:.4f}")
        print(f"   Payout when resolved: $1.0000")
        print(f"   Gross profit: ${gross_edge:.4f} ({gross_edge*100:.2f}%)")
        
        # Estimate net profit after fees
        trade_size = 100  # Assume $100 trade
        fees = trade_size * 0.002 * 2  # 0.2% taker fee on both sides
        gas = 0.05
        slippage = trade_size * 0.001  # 0.1% slippage
        total_costs = fees + gas + slippage
        cost_pct = total_costs / trade_size
        
        net_edge = gross_edge - cost_pct
        
        print(f"\n💸 Cost Estimate (per $100 trade):")
        print(f"   Trading fees: ${fees:.2f}")
        print(f"   Gas: ${gas:.2f}")
        print(f"   Slippage: ${slippage:.2f}")
        print(f"   Total costs: ${total_costs:.2f} ({cost_pct*100:.2f}%)")
        
        print(f"\n💹 Net Profit After Costs:")
        print(f"   Net edge: {net_edge*100:.2f}%")
        print(f"   Profit per $100: ${net_edge*trade_size:.2f}")
        
        if net_edge > 0:
            print(f"\n   ✅ PROFITABLE after all costs!")
        else:
            print(f"\n   ⚠️ Marginal - costs might exceed profit")
    else:
        print(f"\n{'='*60}")
        print(f"❌ NO ARBITRAGE OPPORTUNITIES FOUND")
        print(f"{'='*60}")
        print(f"\n   Checked: First 20 binary markets")
        print(f"   Issue: All spreads too wide (no profitable edge)")
        print(f"\n💡 Suggestions:")
        print(f"   1. Increase market limit in Section 5 (check more markets)")
        print(f"   2. Wait for market conditions to change")
        print(f"   3. Lower minimum edge threshold (riskier)")
        print(f"   4. Try multi-outcome markets (more complex)")
        
        token_ids = None
        selected_market = None

print("\n✅ Section 6 complete!")

In [ ]:
# Diagnostic: Check raw orderbook data

print("\n" + "="*60)
print("🔍 RAW ORDERBOOK DIAGNOSTIC")
print("="*60)

if token_ids and len(token_ids) >= 2:
    print(f"\nFetching orderbooks for both tokens...")
    
    for i, token_id in enumerate(token_ids[:2]):
        print(f"\n{'='*60}")
        print(f"TOKEN {i+1}: {outcomes[i]}")
        print(f"Token ID: {token_id}")
        print(f"{'='*60}")
        
        try:
            ob = client.get_order_book(token_id)
            
            print(f"\n📊 Orderbook object type: {type(ob)}")
            print(f"\n🔑 Available attributes/methods:")
            
            # Show all attributes
            attrs = [attr for attr in dir(ob) if not attr.startswith('_')]
            for attr in attrs[:20]:  # Show first 20
                print(f"   - {attr}")
            
            # Try different access methods
            print(f"\n🔍 Trying different access methods:")
            
            # Method 1: Direct attribute access
            if hasattr(ob, 'asks'):
                print(f"\n   ✅ Has 'asks' attribute")
                print(f"      Type: {type(ob.asks)}")
                if len(ob.asks) > 0:
                    print(f"      Length: {len(ob.asks)}")
                    print(f"      First ask type: {type(ob.asks[0])}")
                    print(f"      First ask: {ob.asks[0]}")
                    
                    # Try to access price
                    first_ask = ob.asks[0]
                    if hasattr(first_ask, 'price'):
                        print(f"      Price (attr): {first_ask.price}")
                    if isinstance(first_ask, dict):
                        print(f"      Price (dict): {first_ask.get('price')}")
            
            if hasattr(ob, 'bids'):
                print(f"\n   ✅ Has 'bids' attribute")
                print(f"      Type: {type(ob.bids)}")
                if len(ob.bids) > 0:
                    print(f"      Length: {len(ob.bids)}")
                    print(f"      First bid: {ob.bids[0]}")
            
            # Method 2: Dict conversion
            if hasattr(ob, 'dict'):
                print(f"\n   ✅ Has 'dict()' method")
                ob_dict = ob.dict()
                print(f"      Keys: {list(ob_dict.keys())}")
                if 'asks' in ob_dict:
                    print(f"      Asks (first 2): {ob_dict['asks'][:2]}")
                if 'bids' in ob_dict:
                    print(f"      Bids (first 2): {ob_dict['bids'][:2]}")
            
            # Method 3: __dict__
            if hasattr(ob, '__dict__'):
                print(f"\n   ✅ Has '__dict__' attribute")
                print(f"      Keys: {list(ob.__dict__.keys())}")
            
        except Exception as e:
            print(f"\n   ❌ Error: {e}")
            import traceback
            print(traceback.format_exc())
else:
    print("\n❌ No token IDs available")

print("\n" + "="*60)

In [ ]:
# Add this right before Section 7 to debug token_ids

print("🔍 DEBUG: Checking token_ids structure")
print(f"Type: {type(token_ids)}")
print(f"Length: {len(token_ids) if token_ids else 0}")

if token_ids:
    print(f"\nFirst 3 tokens:")
    for i, token_id in enumerate(token_ids[:3]):
        print(f"  [{i}] Type: {type(token_id)}, Value: {token_id}")
        print(f"      Length: {len(token_id) if isinstance(token_id, str) else 'N/A'}")
        print(f"      First 50 chars: {str(token_id)[:50]}")

print("\n" + "="*60)

---
## Section 7: Fetch Order Book Data

Get current order books for YES and NO tokens

In [ ]:
print("🔍 Fetching order book data...")
print("="*60)
print("")

if client is None or not token_ids or len(token_ids) < 2:
    print("❌ Cannot fetch order books - no client or token IDs")
    orderbook_data = None
else:
    try:
        print("⏱️ Fetching order books from CLOB...")
        
        # Get order books for each token separately
        yes_token_id = token_ids[0]
        no_token_id = token_ids[1]
        
        print(f"   🟢 Fetching YES token: {yes_token_id[:20]}...")
        yes_orderbook = client.get_order_book(yes_token_id)
        
        print(f"   🔴 Fetching NO token: {no_token_id[:20]}...")
        no_orderbook = client.get_order_book(no_token_id)
        
        # Combine into one structure
        orderbook_data = {
            'yes': yes_orderbook,
            'no': no_orderbook
        }
        
        print("✅ Order book data retrieved!")
        print("")
        
        # Display order books
        print("📊 Order Book Summary:")
        print("")
        
        # YES token
        print("   💹 YES Token:")
        if 'asks' in yes_orderbook and len(yes_orderbook['asks']) > 0:
            best_ask = yes_orderbook['asks'][0]
            print(f"      🟢 Best Ask (Buy): ${best_ask['price']} (Size: {best_ask['size']})")
        else:
            print(f"      ⚠️ No asks available")
            
        if 'bids' in yes_orderbook and len(yes_orderbook['bids']) > 0:
            best_bid = yes_orderbook['bids'][0]
            print(f"      🔴 Best Bid (Sell): ${best_bid['price']} (Size: {best_bid['size']})")
        else:
            print(f"      ⚠️ No bids available")
        print("")
        
        # NO token
        print("   💹 NO Token:")
        if 'asks' in no_orderbook and len(no_orderbook['asks']) > 0:
            best_ask = no_orderbook['asks'][0]
            print(f"      🟢 Best Ask (Buy): ${best_ask['price']} (Size: {best_ask['size']})")
        else:
            print(f"      ⚠️ No asks available")
            
        if 'bids' in no_orderbook and len(no_orderbook['bids']) > 0:
            best_bid = no_orderbook['bids'][0]
            print(f"      🔴 Best Bid (Sell): ${best_bid['price']} (Size: {best_bid['size']})")
        else:
            print(f"      ⚠️ No bids available")
        print("")
        
    except Exception as e:
        print(f"❌ ERROR: {str(e)}")
        import traceback
        print(traceback.format_exc())
        orderbook_data = None

print("")
print("✅ Section 7 complete!")

---
## Section 8: Calculate Arbitrage Opportunity

Check if buying both YES and NO tokens costs less than $1.00

In [ ]:
print("🔍 Calculating arbitrage opportunity...")
print("="*60)
print("")

if orderbook_data is None:
    print("❌ No order book data available")
else:
    try:
        print("⏱️ Analyzing order books...")
        print("")
        
        # Extract prices
        yes_ask = None
        no_ask = None
        
        # Get YES token best ask (price to BUY YES)
        if 'yes' in orderbook_data:
            if 'asks' in orderbook_data['yes'] and len(orderbook_data['yes']['asks']) > 0:
                yes_ask = float(orderbook_data['yes']['asks'][0]['price'])
        
        # Get NO token best ask (price to BUY NO)
        if 'no' in orderbook_data:
            if 'asks' in orderbook_data['no'] and len(orderbook_data['no']['asks']) > 0:
                no_ask = float(orderbook_data['no']['asks'][0]['price'])
        
        if yes_ask is not None and no_ask is not None:
            print("📊 Price Analysis:")
            print(f"   🟢 YES Token Ask: ${yes_ask:.4f}")
            print(f"   🔴 NO Token Ask: ${no_ask:.4f}")
            print("")
            
            # Calculate total cost
            total_cost = yes_ask + no_ask
            print(f"   💰 Total Cost (YES + NO): ${total_cost:.4f}")
            print("")
            
            # Calculate gross edge (before fees)
            gross_edge = 1.0 - total_cost
            gross_edge_bps = gross_edge * 10000
            
            print(f"   📈 Gross Edge: ${gross_edge:.4f} ({gross_edge_bps:.1f} bps)")
            print("")
            
            # Estimate fees (simplified)
            # Polymarket fees vary by market, but typically:
            # - Maker fee: 0% to 0.1%
            # - Taker fee: 0.1% to 0.2%
            # - Gas cost: ~$0.02-0.05 per merge
            
            trade_size = 100.0  # Assume $100 trade
            taker_fee_rate = 0.002  # 0.2% worst case
            total_fees = trade_size * taker_fee_rate * 2  # Both YES and NO orders
            gas_cost = 0.05
            slippage_buffer = trade_size * 0.002  # 0.2%
            
            total_costs = total_fees + gas_cost + slippage_buffer
            costs_per_dollar = total_costs / trade_size
            
            print("   💸 Estimated Costs (per $100 trade):")
            print(f"      - Trading fees: ${total_fees:.2f}")
            print(f"      - Gas cost: ${gas_cost:.2f}")
            print(f"      - Slippage buffer: ${slippage_buffer:.2f}")
            print(f"      - Total: ${total_costs:.2f} ({costs_per_dollar*100:.2f}%)")
            print("")
            
            # Calculate net edge
            net_edge = gross_edge - costs_per_dollar
            net_edge_bps = net_edge * 10000
            
            print(f"   💹 Net Edge (after costs): ${net_edge:.4f} ({net_edge_bps:.1f} bps)")
            print("")
            
            # Profitability check
            if net_edge > 0.01:  # 1% profit threshold
                print("   ✅ PROFITABLE ARBITRAGE DETECTED!")
                print(f"   💰 Estimated profit: ${net_edge * trade_size:.2f} per $100 trade")
            elif net_edge > 0:
                print("   ⚠️ Marginal opportunity (< 1% profit)")
                print(f"   💰 Estimated profit: ${net_edge * trade_size:.2f} per $100 trade")
            else:
                print("   ❌ NOT PROFITABLE (costs exceed edge)")
                print(f"   💸 Would lose: ${abs(net_edge * trade_size):.2f} per $100 trade")
        else:
            print("❌ Could not extract prices from order book")
            print(f"   YES ask: {yes_ask}")
            print(f"   NO ask: {no_ask}")
        
    except Exception as e:
        print(f"❌ ERROR: {str(e)}")
        import traceback
        print(traceback.format_exc())

print("")
print("✅ Section 8 complete!")

---
## Section 9: Fee Calculation Function

Create a reusable function to calculate required edge for profitability

In [ ]:
print("🔍 Creating fee calculation function...")
print("="*60)
print("")

def calculate_min_edge(trade_size_usd, maker_fee=0.001, taker_fee=0.002):
    """
    Calculate minimum edge needed for profitability.
    
    Args:
        trade_size_usd: Dollar amount of trade
        maker_fee: Maker fee rate (default 0.1%)
        taker_fee: Taker fee rate (default 0.2%)
    
    Returns:
        Dictionary with cost breakdown and minimum edge
    """
    print(f"   🔍 Calculating for ${trade_size_usd} trade...")
    
    # Trading fees (worst case: both orders are taker)
    clob_fees = trade_size_usd * taker_fee * 2  # Both YES and NO orders
    
    # Gas cost (relatively fixed on Polygon)
    gas_cost = 0.05
    
    # Slippage (scales with trade size)
    slippage = trade_size_usd * 0.002  # 0.2%
    
    # Total costs
    total = clob_fees + gas_cost + slippage
    
    # As percentage of trade
    min_edge_pct = total / trade_size_usd
    
    # Add 30% safety buffer
    min_edge_safe = min_edge_pct * 1.3
    
    return {
        'trade_size': trade_size_usd,
        'clob_fees': clob_fees,
        'gas_cost': gas_cost,
        'slippage': slippage,
        'total_cost': total,
        'min_edge_pct': min_edge_pct,
        'min_edge_safe': min_edge_safe,
        'min_edge_bps': min_edge_pct * 10000,
        'min_edge_safe_bps': min_edge_safe * 10000
    }

print("✅ Function created!")
print("")

# Test with different trade sizes
print("📊 Testing with different trade sizes:")
print("")

for trade_size in [10, 50, 100, 200]:
    result = calculate_min_edge(trade_size)
    print(f"   💰 ${trade_size} trade:")
    print(f"      Total costs: ${result['total_cost']:.2f}")
    print(f"      Min edge: {result['min_edge_pct']*100:.2f}% ({result['min_edge_bps']:.1f} bps)")
    print(f"      Safe edge: {result['min_edge_safe']*100:.2f}% ({result['min_edge_safe_bps']:.1f} bps)")
    print("")

print("")
print("✅ Section 9 complete!")

---
## Section 10: Final Summary

In [ ]:
print("="*60)
print("🎉 POLYMARKET INTEGRATION TEST COMPLETE!")
print("="*60)
print("")
print("✅ Section 1: Setup & imports - PASSED")
print("✅ Section 2: Configuration loaded - PASSED")
print("✅ Section 3: Private key validated - PASSED" if api_creds_ready else "⚠️ Section 3: Private key - SKIPPED (need private key)")
print("✅ Section 4: Client initialized - PASSED" if client else "⚠️ Section 4: Client - SKIPPED")
print("✅ Section 5: Markets fetched - PASSED" if len(markets) > 0 else "⚠️ Section 5: Markets - SKIPPED")
print("✅ Section 6: Market selected - PASSED" if token_ids else "⚠️ Section 6: Market selection - SKIPPED")
print("✅ Section 7: Order books fetched - PASSED" if orderbook_data else "⚠️ Section 7: Order books - SKIPPED")
print("✅ Section 8: Arbitrage calculated - PASSED" if orderbook_data else "⚠️ Section 8: Arbitrage - SKIPPED")
print("✅ Section 9: Fee calculator created - PASSED")
print("")
print("📋 What We've Learned:")
print("   ✅ How to connect to Polymarket CLOB")
print("   ✅ How to fetch market data")
print("   ✅ How to extract token IDs from markets")
print("   ✅ How to analyze order books")
print("   ✅ How to calculate arbitrage opportunities")
print("   ✅ How to estimate costs and profitability")
print("")
print("📋 Next Steps:")
print("   1. ✅ Private key configured")
print("   2. Fund wallet with USDC for live trading")
print("   3. Build WebSocket streaming for real-time data")
print("   4. Implement automated order placement")
print("   5. Add risk management and monitoring")
print("")
print("✅ Ready to build the production bot!")